# Vehicle Counting
Implementasi vehicle counting menggunakan algoritma blobDetection

Oleh :
- Ahmad Elang
- Aji Imawan Omi
- Faridah Nur Suci
- Nur Intan 
- Novianti Aliansih

Modification from : alexdrake TrafficCounter/blobDetection.py

In [5]:
import cv2
import numpy as np
import time
import logging
import math
import re
from os import walk
import os

### Class Vehicle
Kelas Vehicle yang terdiri dari id, posisi, frame yang terakhir dilihat, frame yang sedang dilihat, boolean isCounted dan vehicle direction.

In [6]:
class Vehicle(object):
    def __init__(self, id, position):
        self.id = id
        self.positions = [position]
        self.frames_since_seen = 0
        self.frames_seen = 0
        self.counted = False

    @property
    def last_position(self):
        return self.positions[-1]
    
    @property
    def last_position2(self):
        return self.positions[-2]

    def add_position(self, new_position):
        self.positions.append(new_position)
        self.frames_since_seen = 0
        self.frames_seen += 1

    def draw(self, output_image):
        for point in self.positions:
            cv2.circle(output_image, point, 2, (0, 0, 255), -1)
            cv2.polylines(output_image, [np.int32(self.positions)]
                , False, (0, 0, 255), 1)

### Class Vehicle Counter
Kelas yang menghandle tracking dan counting vehicle dengan cara menghitung jarak vehicle yang bergerak setiap framenya

In [7]:
class VehicleCounter(object):
    def __init__(self, shape):
        self.log = logging.getLogger("vehicle_counter")
        
        self.height, self.width = shape

        self.vehicles = []
        self.next_vehicle_id = 0
        self.vehicle_count = 0
        self.vehicle_LHS = 0
        self.vehicle_RHS = 0
        self.max_unseen_frames = 10


    @staticmethod
    def get_vector(a, b):
        """Calculate vector (distance, angle in degrees) from point a to point b.

        Angle ranges from -180 to 180 degrees.
        Vector with angle 0 points straight down on the image.
        Values decrease in clockwise direction.
        """
        dx = float(b[0] - a[0])
        dy = float(b[1] - a[1])

        distance = math.sqrt(dx**2 + dy**2)

        if dy > 0:
            angle = math.degrees(math.atan(-dx/dy))
        elif dy == 0:
            if dx < 0:
                angle = 90.0
            elif dx > 0:
                angle = -90.0
            else:
                angle = 0.0
        else:
            if dx < 0:
                angle = 180 - math.degrees(math.atan(dx/dy))
            elif dx > 0:
                angle = -180 - math.degrees(math.atan(dx/dy))
            else:
                angle = 180.0        

        return distance, angle, dx, dy 


    @staticmethod
    def is_valid_vector(a, b):
        # vector is only valid if threshold distance is less than 12
        # and if vector deviation is less than 30 or greater than 330 degs
        distance, angle, _, _ = a
        threshold_distance = 12.0
        return (distance <= threshold_distance)


    def update_vehicle(self, vehicle, matches):
        # Find if any of the matches fits this vehicle
        for i, match in enumerate(matches):
            contour, centroid = match
            
            # store the vehicle data
            vector = self.get_vector(vehicle.last_position, centroid)
            
            # only measure angle deviation if we have enough points
            if vehicle.frames_seen > 2:
                prevVector = self.get_vector(vehicle.last_position2, vehicle.last_position)
                angleDev = abs(prevVector[1]-vector[1])
            else:
                angleDev = 0
                
            b = dict(
                    id = vehicle.id,
                    center_x = centroid[0],
                    center_y = centroid[1],
                    vector_x = vector[0],
                    vector_y = vector[1],
                    dx = vector[2],
                    dy = vector[3],
                    counted = vehicle.counted,
                    frame_number = frame_no,
                    angle_dev = angleDev
                    )
            
            tracked_blobs.append(b)
            
            # check validity
            if self.is_valid_vector(vector, angleDev):    
                vehicle.add_position(centroid)
                vehicle.frames_seen += 1
                self.log.debug("Added match (%d, %d) to vehicle #%d. vector=(%0.2f,%0.2f)"
                    , centroid[0], centroid[1], vehicle.id, vector[0], vector[1])
                return i

        # No matches fit...        
        vehicle.frames_since_seen += 1
        self.log.debug("No match for vehicle #%d. frames_since_seen=%d"
            , vehicle.id, vehicle.frames_since_seen)

        return None


    def update_count(self, matches, output_image = None):
        self.log.debug("Updating count using %d matches...", len(matches))

        # First update all the existing vehicles
        for vehicle in self.vehicles:
            i = self.update_vehicle(vehicle, matches)
            if i is not None:
                del matches[i]

        # Add new vehicles based on the remaining matches
        for match in matches:
            contour, centroid = match
            new_vehicle = Vehicle(self.next_vehicle_id, centroid)
            self.next_vehicle_id += 1
            self.vehicles.append(new_vehicle)
            self.log.debug("Created new vehicle #%d from match (%d, %d)."
                , new_vehicle.id, centroid[0], centroid[1])

        # Count any uncounted vehicles that are past the divider
        for vehicle in self.vehicles:
            if not vehicle.counted and (vehicle.frames_seen > 6):

                vehicle.counted = True
                # update appropriate counter
                if (vehicle.last_position[0] >= (int(frame_w/2)-10)):
                    self.vehicle_count += 1
                elif (vehicle.last_position[0] <= (int(frame_w/2)+10)):
                    self.vehicle_count += 1
                    
                self.log.debug("Counted vehicle #%d (total count=%d)."
                    , vehicle.id, self.vehicle_count)

        # Optionally draw the vehicles on an image
        if output_image is not None:
            for vehicle in self.vehicles:
                vehicle.draw(output_image)
                
            # count vehicle
            cv2.putText(output_image, ("Vehicle counted: %02d" % self.vehicle_count), (12, 56)
                , cv2.FONT_HERSHEY_PLAIN, 1.2, (127,255, 255), 2)
            
        # Remove vehicles that have not been seen long enough
        removed = [ v.id for v in self.vehicles
            if v.frames_since_seen >= self.max_unseen_frames ]
        self.vehicles[:] = [ v for v in self.vehicles
            if not v.frames_since_seen >= self.max_unseen_frames ]
        for id in removed:
            self.log.debug("Removed vehicle #%d.", id)

        self.log.debug("Count updated, tracking %d vehicles.", len(self.vehicles))

### Main Program

In [ ]:
loc = os.path.abspath('')

# Video source
inputFile = loc+'/inputs/1300/UI_Pengambilan2_01.avi'

# for testing
tracked_blobs = []
tracked_conts = []
t_retval = []

cap = cv2.VideoCapture(inputFile)
f = []

avg = None

# get frame size
# python 2
# frame_w = int(cap.get(cv2.cv.CV_CAP_PROP_FRAME_WIDTH))
# frame_h = int(cap.get(cv2.cv.CV_CAP_PROP_FRAME_HEIGHT))

# python 3
frame_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# create a mask (manual for each camera)
mask = np.zeros((frame_h,frame_w), np.uint8)
mask[:,:] = 255
mask[:100, :] = 0
mask[230:, 160:190] = 0
mask[170:230,170:190] = 0
mask[140:170,176:190] = 0
mask[100:140,176:182] = 0

# The cutoff for threshold. A lower number means smaller changes between
# the average and current scene are more readily detected.
THRESHOLD_SENSITIVITY = 40
t_retval.append(THRESHOLD_SENSITIVITY)
# Blob size limit before we consider it for tracking.
CONTOUR_WIDTH = 21
CONTOUR_HEIGHT = 16#21

# The weighting to apply to "this" frame when averaging. A higher number
# here means that the average scene will pick up changes more readily,
# thus making the difference between average and current scenes smaller.
DEFAULT_AVERAGE_WEIGHT = 0.01
INITIAL_AVERAGE_WEIGHT = DEFAULT_AVERAGE_WEIGHT / 50
# Blob smoothing function, to join 'gaps' in cars
SMOOTH = max(2,int(round((CONTOUR_WIDTH**0.5)/2,0)))
# Constants for drawing on the frame.
LINE_THICKNESS = 1

# python 2
# fourcc = cv2.cv.CV_FOURCC(*'mp4v')
# python 3
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

out = loc+'/outputs/output.mp4'
out = cv2.VideoWriter(out, fourcc, 20, (frame_w, frame_h))

outblob = loc+'/outputs/_outblob.mp4'
diffop = loc+'/outputs/_outdiff.mp4'
outblob = cv2.VideoWriter(outblob, fourcc, 20, (frame_w, frame_h))
diffop = cv2.VideoWriter(diffop, fourcc, 20, (frame_w, frame_h))

# A list of "tracked blobs".
blobs = []
car_counter = None  # will be created later
frame_no = 0

# python 2
# total_frames = cap.get(cv2.cv.CV_CAP_PROP_FRAME_COUNT)
# python 3
total_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

total_cars = 0

start_time = time.time()
ret, frame = cap.read()

while ret:    
    ret, frame = cap.read()
    frame_no = frame_no + 1
    
    if ret and frame_no < total_frames:

        print("Processing frame ",frame_no)
        
        # get returned time
        frame_time = time.time()
        
        # convert BGR to HSV
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        
        # only use the Value channel of the frame
        (_,_,grayFrame) = cv2.split(frame)
        grayFrame = cv2.bilateralFilter(grayFrame, 11, 21, 21)

        if avg is None:
            # Set up the average if this is the first time through.
            avg = grayFrame.copy().astype("float")
            continue
        
        # Build the average scene image by accumulating this frame
        # with the existing average.
        if frame_no < 10:
            def_wt = INITIAL_AVERAGE_WEIGHT
        else:
            def_wt = DEFAULT_AVERAGE_WEIGHT
            
        cv2.accumulateWeighted(grayFrame, avg, def_wt)
        
        if frame_no > int(200):
            grayOp = cv2.cvtColor(cv2.convertScaleAbs(avg), cv2.COLOR_GRAY2BGR)
            backOut = loc+"/backgrounds/bg.jpg"
            cv2.imwrite(backOut, grayOp)
            
     # Compute the grayscale difference between the current grayscale frame and
        # the average of the scene.
        differenceFrame = cv2.absdiff(grayFrame, cv2.convertScaleAbs(avg))
        # blur the difference image
        differenceFrame = cv2.GaussianBlur(differenceFrame, (5, 5), 0)

        diffout = cv2.cvtColor(differenceFrame, cv2.COLOR_GRAY2BGR)
        diffop.write(diffout)

        # get estimated otsu threshold level
        retval, _ = cv2.threshold(differenceFrame, 0, 255,
                                  cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        # add to list of threshold levels
        t_retval.append(retval)
        
        # apply threshold based on average threshold value
        if frame_no < 10:
            ret2, thresholdImage = cv2.threshold(differenceFrame, 
                                                 int(np.mean(t_retval)*0.9),
                                                 255, cv2.THRESH_BINARY)
        else:
            ret2, thresholdImage = cv2.threshold(differenceFrame, 
                                             int(np.mean(t_retval[-10:-1])*0.9),
                                             255, cv2.THRESH_BINARY)
        
        # We'll need to fill in the gaps to make a complete vehicle as windows
        # and other features can split them!
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (SMOOTH, SMOOTH))
        # Fill any small holes
        thresholdImage = cv2.morphologyEx(thresholdImage, cv2.MORPH_CLOSE, kernel)
        
        # Remove noise
        thresholdImage = cv2.morphologyEx(thresholdImage, cv2.MORPH_OPEN, kernel)

        # Dilate to merge adjacent blobs
        thresholdImage = cv2.dilate(thresholdImage, kernel, iterations = 2)
        
        # apply mask
        thresholdImage = cv2.bitwise_and(thresholdImage, thresholdImage, mask = mask)

        threshout = cv2.cvtColor(thresholdImage, cv2.COLOR_GRAY2BGR)
        outblob.write(threshout)
        
        # python 2
        # Find contours aka blobs in the threshold image.
        #contours, hierarchy = cv2.findContours(thresholdImage, 
        #                                         cv2.RETR_EXTERNAL, 
        #                                         cv2.CHAIN_APPROX_SIMPLE)
        # python 3
        _, contours, _ = cv2.findContours(thresholdImage, 
                                                  cv2.RETR_EXTERNAL, 
                                                  cv2.CHAIN_APPROX_SIMPLE)
        
        print("Found ",len(contours)," vehicle contours.")
        # process contours if they exist!
        if contours:
            for (i, contour) in enumerate(contours):    
                # Find the bounding rectangle and center for each blob
                (x, y, w, h) = cv2.boundingRect(contour)
                contour_valid = (w > CONTOUR_WIDTH) and (h > CONTOUR_HEIGHT)
                
                print("Contour #",i,": pos=(x=",x,", y=",y,") size=(w=",w,
                      ", h=",h,") valid=",contour_valid)
                
                if not contour_valid:
                    continue
                
                center = (int(x + w/2), int(y + h/2))
                blobs.append(((x, y, w, h), center))
                    
        for (i, match) in enumerate(blobs):
            contour, centroid = match
            x, y, w, h = contour
            
            # store the contour data
            c = dict(
                        frame_no = frame_no,
                        centre_x = x,
                        centre_y = y,
                        width = w,
                        height = h
                        )
            tracked_conts.append(c)
            
            cv2.rectangle(frame, (x, y), (x + w - 1, y + h - 1), (0, 0, 255), LINE_THICKNESS)
            cv2.circle(frame, centroid, 2, (0, 0, 255), -1)
        
        if car_counter is None:
            print("Creating vehicle counter...")
            car_counter = VehicleCounter(frame.shape[:2])
            
        # get latest count
        car_counter.update_count(blobs, frame)
        current_count = car_counter.vehicle_RHS + car_counter.vehicle_LHS
        
        # print elapsed time to console
        elapsed_time = time.time()-start_time
        print("-- %s seconds --" % round(elapsed_time,2))
        
        # output video
        frame = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)
        
        # draw dividing line
        # flash green when new car counted
        if current_count > total_cars:
            cv2.line(frame, (0, int(frame_h/2)),(frame_w, int(frame_h/2)),
                 (0,255,0), 2*LINE_THICKNESS)
        else:
            cv2.line(frame, (0, int(frame_h/2)),(frame_w, int(frame_h/2)),
             (0,0,255), LINE_THICKNESS)
            
         # update with latest count
        total_cars = current_count  
        
        # draw upper limit
        cv2.line(frame, (0, 100),(frame_w, 100), (0,0,0), LINE_THICKNESS)
        
        cv2.imshow("preview", frame)
        out.write(frame)
        
        if cv2.waitKey(27) and 0xFF == ord('q'):
            break
    else:
        break

cv2.destroyAllWindows()
cap.release()
out.release()



Processing frame  1
Processing frame  2
Found  0  vehicle contours.
Creating vehicle counter...
-- 0.06 seconds --
Processing frame  3
Found  0  vehicle contours.
-- 1.78 seconds --
Processing frame  4
Found  0  vehicle contours.
-- 1.84 seconds --
Processing frame  5
Found  1  vehicle contours.
Contour # 0 : pos=(x= 295 , y= 123 ) size=(w= 5 , h= 6 ) valid= False
-- 1.91 seconds --
Processing frame  6
Found  1  vehicle contours.
Contour # 0 : pos=(x= 253 , y= 200 ) size=(w= 16 , h= 15 ) valid= False
-- 1.98 seconds --
Processing frame  7
Found  1  vehicle contours.
Contour # 0 : pos=(x= 253 , y= 200 ) size=(w= 16 , h= 15 ) valid= False
-- 2.04 seconds --
Processing frame  8
Found  1  vehicle contours.
Contour # 0 : pos=(x= 253 , y= 201 ) size=(w= 15 , h= 14 ) valid= False
-- 2.1 seconds --
Processing frame  9
Found  1  vehicle contours.
Contour # 0 : pos=(x= 253 , y= 201 ) size=(w= 15 , h= 14 ) valid= False
-- 2.17 seconds --
Processing frame  10
Found  1  vehicle contours.
Contour # 

Processing frame  48
Found  5  vehicle contours.
Contour # 0 : pos=(x= 253 , y= 204 ) size=(w= 15 , h= 11 ) valid= False
Contour # 1 : pos=(x= 23 , y= 136 ) size=(w= 5 , h= 5 ) valid= False
Contour # 2 : pos=(x= 13 , y= 131 ) size=(w= 4 , h= 4 ) valid= False
Contour # 3 : pos=(x= 28 , y= 117 ) size=(w= 4 , h= 4 ) valid= False
Contour # 4 : pos=(x= 0 , y= 117 ) size=(w= 5 , h= 4 ) valid= False
-- 4.83 seconds --
Processing frame  49
Found  5  vehicle contours.
Contour # 0 : pos=(x= 253 , y= 204 ) size=(w= 15 , h= 11 ) valid= False
Contour # 1 : pos=(x= 23 , y= 136 ) size=(w= 5 , h= 5 ) valid= False
Contour # 2 : pos=(x= 13 , y= 131 ) size=(w= 4 , h= 4 ) valid= False
Contour # 3 : pos=(x= 28 , y= 117 ) size=(w= 4 , h= 4 ) valid= False
Contour # 4 : pos=(x= 0 , y= 117 ) size=(w= 5 , h= 4 ) valid= False
-- 4.89 seconds --
Processing frame  50
Found  6  vehicle contours.
Contour # 0 : pos=(x= 253 , y= 204 ) size=(w= 15 , h= 11 ) valid= False
Contour # 1 : pos=(x= 23 , y= 136 ) size=(w= 5 , 

Processing frame  66
Found  8  vehicle contours.
Contour # 0 : pos=(x= 253 , y= 204 ) size=(w= 15 , h= 11 ) valid= False
Contour # 1 : pos=(x= 23 , y= 136 ) size=(w= 5 , h= 6 ) valid= False
Contour # 2 : pos=(x= 16 , y= 135 ) size=(w= 4 , h= 4 ) valid= False
Contour # 3 : pos=(x= 0 , y= 116 ) size=(w= 6 , h= 9 ) valid= False
Contour # 4 : pos=(x= 27 , y= 115 ) size=(w= 5 , h= 6 ) valid= False
Contour # 5 : pos=(x= 0 , y= 108 ) size=(w= 6 , h= 5 ) valid= False
Contour # 6 : pos=(x= 32 , y= 103 ) size=(w= 5 , h= 5 ) valid= False
Contour # 7 : pos=(x= 0 , y= 100 ) size=(w= 10 , h= 5 ) valid= False
-- 6.1 seconds --
Processing frame  67
Found  8  vehicle contours.
Contour # 0 : pos=(x= 253 , y= 204 ) size=(w= 15 , h= 11 ) valid= False
Contour # 1 : pos=(x= 23 , y= 136 ) size=(w= 5 , h= 6 ) valid= False
Contour # 2 : pos=(x= 16 , y= 135 ) size=(w= 4 , h= 4 ) valid= False
Contour # 3 : pos=(x= 0 , y= 116 ) size=(w= 6 , h= 9 ) valid= False
Contour # 4 : pos=(x= 27 , y= 115 ) size=(w= 5 , h= 6

Processing frame  81
Found  9  vehicle contours.
Contour # 0 : pos=(x= 254 , y= 205 ) size=(w= 14 , h= 10 ) valid= False
Contour # 1 : pos=(x= 24 , y= 136 ) size=(w= 4 , h= 5 ) valid= False
Contour # 2 : pos=(x= 16 , y= 135 ) size=(w= 4 , h= 4 ) valid= False
Contour # 3 : pos=(x= 27 , y= 115 ) size=(w= 6 , h= 6 ) valid= False
Contour # 4 : pos=(x= 0 , y= 114 ) size=(w= 7 , h= 12 ) valid= False
Contour # 5 : pos=(x= 54 , y= 111 ) size=(w= 5 , h= 4 ) valid= False
Contour # 6 : pos=(x= 17 , y= 108 ) size=(w= 4 , h= 4 ) valid= False
Contour # 7 : pos=(x= 21 , y= 100 ) size=(w= 5 , h= 4 ) valid= False
Contour # 8 : pos=(x= 0 , y= 100 ) size=(w= 10 , h= 13 ) valid= False
-- 7.19 seconds --
Processing frame  82
Found  10  vehicle contours.
Contour # 0 : pos=(x= 254 , y= 205 ) size=(w= 14 , h= 10 ) valid= False
Contour # 1 : pos=(x= 23 , y= 135 ) size=(w= 5 , h= 6 ) valid= False
Contour # 2 : pos=(x= 16 , y= 135 ) size=(w= 4 , h= 4 ) valid= False
Contour # 3 : pos=(x= 27 , y= 115 ) size=(w= 6 

Processing frame  96
Found  5  vehicle contours.
Contour # 0 : pos=(x= 254 , y= 205 ) size=(w= 14 , h= 10 ) valid= False
Contour # 1 : pos=(x= 27 , y= 115 ) size=(w= 6 , h= 6 ) valid= False
Contour # 2 : pos=(x= 0 , y= 114 ) size=(w= 7 , h= 8 ) valid= False
Contour # 3 : pos=(x= 0 , y= 107 ) size=(w= 7 , h= 6 ) valid= False
Contour # 4 : pos=(x= 0 , y= 100 ) size=(w= 8 , h= 5 ) valid= False
-- 8.27 seconds --
Processing frame  97
Found  5  vehicle contours.
Contour # 0 : pos=(x= 254 , y= 205 ) size=(w= 14 , h= 10 ) valid= False
Contour # 1 : pos=(x= 27 , y= 115 ) size=(w= 6 , h= 6 ) valid= False
Contour # 2 : pos=(x= 0 , y= 114 ) size=(w= 7 , h= 8 ) valid= False
Contour # 3 : pos=(x= 0 , y= 107 ) size=(w= 7 , h= 6 ) valid= False
Contour # 4 : pos=(x= 0 , y= 100 ) size=(w= 8 , h= 5 ) valid= False
-- 8.34 seconds --
Processing frame  98
Found  5  vehicle contours.
Contour # 0 : pos=(x= 254 , y= 205 ) size=(w= 14 , h= 9 ) valid= False
Contour # 1 : pos=(x= 28 , y= 116 ) size=(w= 4 , h= 5 

...

Found  6  vehicle contours.
Contour # 0 : pos=(x= 260 , y= 209 ) size=(w= 6 , h= 5 ) valid= False
Contour # 1 : pos=(x= 25 , y= 193 ) size=(w= 5 , h= 5 ) valid= False
Contour # 2 : pos=(x= 0 , y= 124 ) size=(w= 5 , h= 5 ) valid= False
Contour # 3 : pos=(x= 219 , y= 118 ) size=(w= 30 , h= 34 ) valid= True
Contour # 4 : pos=(x= 189 , y= 100 ) size=(w= 7 , h= 6 ) valid= False
Contour # 5 : pos=(x= 0 , y= 100 ) size=(w= 7 , h= 6 ) valid= False
-- 246.64 seconds --
Processing frame  3439
Found  6  vehicle contours.
Contour # 0 : pos=(x= 260 , y= 209 ) size=(w= 6 , h= 5 ) valid= False
Contour # 1 : pos=(x= 24 , y= 193 ) size=(w= 6 , h= 6 ) valid= False
Contour # 2 : pos=(x= 0 , y= 124 ) size=(w= 5 , h= 5 ) valid= False
Contour # 3 : pos=(x= 219 , y= 118 ) size=(w= 30 , h= 34 ) valid= True
Contour # 4 : pos=(x= 189 , y= 100 ) size=(w= 7 , h= 6 ) valid= False
Contour # 5 : pos=(x= 0 , y= 100 ) size=(w= 7 , h= 7 ) valid= False
-- 246.72 seconds --
Processing frame  3440
Found  5  vehicle contou

Found  5  vehicle contours.
Contour # 0 : pos=(x= 26 , y= 194 ) size=(w= 4 , h= 5 ) valid= False
Contour # 1 : pos=(x= 224 , y= 127 ) size=(w= 33 , h= 39 ) valid= True
Contour # 2 : pos=(x= 0 , y= 124 ) size=(w= 5 , h= 4 ) valid= False
Contour # 3 : pos=(x= 190 , y= 100 ) size=(w= 9 , h= 17 ) valid= False
Contour # 4 : pos=(x= 4 , y= 100 ) size=(w= 2 , h= 2 ) valid= False
-- 247.96 seconds --
Processing frame  3457
Found  5  vehicle contours.
Contour # 0 : pos=(x= 26 , y= 194 ) size=(w= 4 , h= 5 ) valid= False
Contour # 1 : pos=(x= 224 , y= 127 ) size=(w= 33 , h= 39 ) valid= True
Contour # 2 : pos=(x= 0 , y= 124 ) size=(w= 5 , h= 4 ) valid= False
Contour # 3 : pos=(x= 190 , y= 100 ) size=(w= 9 , h= 17 ) valid= False
Contour # 4 : pos=(x= 4 , y= 100 ) size=(w= 2 , h= 2 ) valid= False
-- 248.03 seconds --
Processing frame  3458
Found  5  vehicle contours.
Contour # 0 : pos=(x= 26 , y= 194 ) size=(w= 4 , h= 4 ) valid= False
Contour # 1 : pos=(x= 224 , y= 128 ) size=(w= 34 , h= 40 ) valid=

Found  4  vehicle contours.
Contour # 0 : pos=(x= 232 , y= 145 ) size=(w= 39 , h= 47 ) valid= True
Contour # 1 : pos=(x= 193 , y= 113 ) size=(w= 10 , h= 26 ) valid= False
Contour # 2 : pos=(x= 200 , y= 100 ) size=(w= 7 , h= 6 ) valid= False
Contour # 3 : pos=(x= 186 , y= 100 ) size=(w= 3 , h= 1 ) valid= False
-- 249.91 seconds --
Processing frame  3484
Found  4  vehicle contours.
Contour # 0 : pos=(x= 233 , y= 147 ) size=(w= 40 , h= 48 ) valid= True
Contour # 1 : pos=(x= 193 , y= 115 ) size=(w= 11 , h= 26 ) valid= False
Contour # 2 : pos=(x= 199 , y= 100 ) size=(w= 8 , h= 7 ) valid= False
Contour # 3 : pos=(x= 184 , y= 100 ) size=(w= 12 , h= 2 ) valid= False
-- 249.99 seconds --
Processing frame  3485
Found  4  vehicle contours.
Contour # 0 : pos=(x= 233 , y= 147 ) size=(w= 40 , h= 48 ) valid= True
Contour # 1 : pos=(x= 193 , y= 115 ) size=(w= 11 , h= 26 ) valid= False
Contour # 2 : pos=(x= 199 , y= 100 ) size=(w= 8 , h= 7 ) valid= False
Contour # 3 : pos=(x= 184 , y= 100 ) size=(w= 12

Found  3  vehicle contours.
Contour # 0 : pos=(x= 246 , y= 173 ) size=(w= 48 , h= 41 ) valid= True
Contour # 1 : pos=(x= 197 , y= 144 ) size=(w= 14 , h= 33 ) valid= False
Contour # 2 : pos=(x= 183 , y= 100 ) size=(w= 30 , h= 24 ) valid= True
-- 252.17 seconds --
Processing frame  3514
Found  3  vehicle contours.
Contour # 0 : pos=(x= 247 , y= 175 ) size=(w= 48 , h= 39 ) valid= True
Contour # 1 : pos=(x= 198 , y= 147 ) size=(w= 14 , h= 34 ) valid= False
Contour # 2 : pos=(x= 183 , y= 100 ) size=(w= 30 , h= 25 ) valid= True
-- 252.25 seconds --
Processing frame  3515
Found  3  vehicle contours.
Contour # 0 : pos=(x= 247 , y= 175 ) size=(w= 48 , h= 39 ) valid= True
Contour # 1 : pos=(x= 198 , y= 147 ) size=(w= 13 , h= 34 ) valid= False
Contour # 2 : pos=(x= 183 , y= 100 ) size=(w= 30 , h= 25 ) valid= True
-- 252.33 seconds --
Processing frame  3516
Found  3  vehicle contours.
Contour # 0 : pos=(x= 248 , y= 177 ) size=(w= 48 , h= 37 ) valid= True
Contour # 1 : pos=(x= 198 , y= 150 ) size=(

Processing frame  3537
Found  5  vehicle contours.
Contour # 0 : pos=(x= 263 , y= 205 ) size=(w= 35 , h= 9 ) valid= False
Contour # 1 : pos=(x= 205 , y= 187 ) size=(w= 17 , h= 27 ) valid= False
Contour # 2 : pos=(x= 185 , y= 110 ) size=(w= 34 , h= 34 ) valid= True
Contour # 3 : pos=(x= 188 , y= 100 ) size=(w= 17 , h= 9 ) valid= False
Contour # 4 : pos=(x= 0 , y= 100 ) size=(w= 5 , h= 5 ) valid= False
-- 254.14 seconds --
Processing frame  3538
Found  5  vehicle contours.
Contour # 0 : pos=(x= 266 , y= 208 ) size=(w= 29 , h= 6 ) valid= False
Contour # 1 : pos=(x= 206 , y= 187 ) size=(w= 17 , h= 27 ) valid= False
Contour # 2 : pos=(x= 186 , y= 112 ) size=(w= 34 , h= 34 ) valid= True
Contour # 3 : pos=(x= 188 , y= 100 ) size=(w= 18 , h= 11 ) valid= False
Contour # 4 : pos=(x= 0 , y= 100 ) size=(w= 5 , h= 5 ) valid= False
-- 254.23 seconds --
Processing frame  3539
Found  5  vehicle contours.
Contour # 0 : pos=(x= 266 , y= 208 ) size=(w= 29 , h= 6 ) valid= False
Contour # 1 : pos=(x= 206 ,